## Get average trade price for each coin of interest over trade history

In [ ]:
import sys
sys.version_info

In [ ]:
import numpy as np
import pandas as pd
import pickle as pk

#### Import files

In [ ]:
with open('../data/coins_of_interest.txt', 'r') as f:
    coins_of_interest = list(f.read().split('\n'))
    
with open('../data/withdrawal.pk', 'rb') as f:
    withdrawal = pk.load(f)

#### Subset trade_history dataframe to only coins of interest

In [ ]:
with open('../data/trade_history_converted.pk', 'rb') as f: 
    trade_history = pk.load(f) 
    trade_history.price = pd.to_numeric(trade_history.price)
    trade_history.qty = pd.to_numeric(trade_history.qty)
    trade_history.commission = pd.to_numeric(trade_history.commission)

In [ ]:
#df = trade_history
df = trade_history[trade_history.baseAsset.isin(coins_of_interest)]
print(df.shape)

In [ ]:
print(df[df.baseAsset=='DASH'][['symbol','price','time','isBuyer','baseAsset', 'quoteAsset','baseAssetBTC']])

In [ ]:
#Check column datatypes 
df.dtypes

In [ ]:
withdrawal[:5]

## Calculation
`summary: <dataframe>`
- `qty`: total number of coins after commission fees are deducted
- `costBTC`: BTC value invested in the coin
- `costUSDT`: USDT value invested in the coin
- `buyBTC`: average BTC price of coin bought
- `sellBTC`: average BTC price of coin sold
- `buyUSDT`: average USDT price of coin bought
- `sellUSDT`: average USDT price of coin sold

In [ ]:
df.loc['tradeBTC'] = df.apply(lambda x: x.qty*x.baseAssetBTC+x.commissionAssetBTC*x.commission, axis =1)
df.loc['tradeUSDT'] = df.apply(lambda x: x.baseAssetUSDT*x.qty+x.commissionAssetUSDT*x.commission, axis =1)

summary = pd.DataFrame()
#summary['buyBTC'] = df[df.isBuyer == True][['baseAsset','tradeBTC', 'qty']].groupby('baseAsset').transform(lambda x: sum(x.tr))
#summary['buyUSDT'] = df[df.isBuyer == True][['baseAsset','tradeUSDT', 'qty']].groupby('baseAsset').agg('sum')



#### Calculate quantity of coins
qty = trades + commission fee + withdrawal fee

In [ ]:
qty_dict = {}

#trades
for name, group in df.groupby('baseAsset'):
    
    qty_ls = group.apply(lambda x: -x.qty if x.isBuyer == False else x.qty, axis =1)
    qty_dict[name] = [np.sum(qty_ls)]

#commission fee
for name, group in df.groupby('commissionAsset'):
    try:
        qty_ls = group.apply(lambda x: -x.commission, axis =1)
        qty_dict[name].append(np.sum(qty_ls))
    except KeyError:
        qty_dict[name] = [np.sum(qty_ls)]
        
summary['tradeqty'] = [sum(qty_dict.get(i)) for i in summary.index]

#withdrawal fee
for name, group in withdrawal.groupby('asset'):
    try:
        qty_ls = group.apply(lambda x: -x.transactionFee, axis =1)
        qty_dict[name].append(np.sum(qty_ls))
    except KeyError:
        qty_dict[name] = [np.sum(qty_ls)]
        
for key, value in qty_dict.items():
    qty_dict[key] = np.sum(value)

summary['qty'] = [qty_dict.get(i) for i in summary.index]  

In [ ]:
summary['buyBTC'] = \
(
    df[df.isBuyer == True].groupby('baseAsset')
    .pipe(lambda grp: grp.tradeBTC.sum()/grp.qty.sum())
)
summary['buyUSDT'] = \
(
    df[df.isBuyer == True].groupby('baseAsset')
    .pipe(lambda grp: grp.tradeUSDT.sum()/grp.qty.sum())
)
summary['sellBTC'] = \
(
    df[df.isBuyer == False].groupby('baseAsset')
    .pipe(lambda grp: grp.tradeBTC.sum()/grp.qty.sum())
)
summary['sellUSDT'] = \
(
    df[df.isBuyer == False].groupby('baseAsset')
    .pipe(lambda grp: grp.tradeUSDT.sum()/grp.qty.sum())
)

summary['costBTC'] = summary.tradeqty*summary.buyBTC

summary['costUSDT'] = summary.tradeqty*summary.buyUSDT

In [ ]:
summary.to_csv("../data/summary.csv")